In [4]:
import pandas as pd
import sys
sys.path.append('../src')
from datasets import load_dataset
from datasets import ClassLabel, Sequence
from DataLoader import DataLoader

In [2]:
prepared = pd.read_csv('../output/prepared_data.csv')
DataLoader(prepared, test_ratio=.1).save_to_csv('../output')
prepared

,sentence_id,language,intent,written,spoken,type,token_id,token,tag
0,0,en,OpenArea,show,show,plain,0,show,O
1,0,en,OpenArea,photo,photo,mycloudarea,1,photo,O
2,0,en,OpenArea,on,on,plain,2,on,O
3,0,en,OpenArea,mycloud,mycloud,plain,3,mycloud,O
4,1,en,OpenArea,show,show,plain,0,show,O
...,...,...,...,...,...,...,...,...,...
7609,1364,en,WatchNextEpisode,another,another,plain,0,another,O
7610,1364,en,WatchNextEpisode,episode,episode,plain,1,episode,O
7611,1365,en,WatchNextEpisode,one,one,plain,0,one,O
7612,1365,en,WatchNextEpisode,more,more,plain,1,more,O


In [47]:
from datasets import Dataset, DatasetDict
def read_dataset_from_csv(path):
    df = pd.read_csv(path)
    feature_tag = Sequence(ClassLabel(num_classes=3, names=list(pd.factorize(df['tag'])[1])))
    df['tag'] = df['tag'].apply(feature_tag.feature.str2int)
    df_text = df.groupby(['sentence_id']).agg({'token':list, 'tag':list})
    dataset = Dataset.from_pandas(df_text)
    dataset.features[f"tag"] = feature_tag
    return dataset

DatasetDict({
    train: Dataset({
        features: ['token', 'tag', 'sentence_id'],
        num_rows: 1106
    })
    test: Dataset({
        features: ['token', 'tag', 'sentence_id'],
        num_rows: 136
    })
    validation: Dataset({
        features: ['token', 'tag', 'sentence_id'],
        num_rows: 123
    })
    train_all: Dataset({
        features: ['token', 'tag', 'sentence_id'],
        num_rows: 1229
    })
})

In [48]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

show_random_elements(datasets["train"])


,token,tag,sentence_id
0,"[i, would, like, to, listen, to, drs, four]","[O, O, O, O, O, O, B-TBNorm, I-TBNorm]",179
1,"[is, it, pouring]","[O, O, O]",1170
2,"[volume, down, please]","[O, O, O]",1097
3,"[give, me, some, information, about, richard, gere]","[O, O, O, O, O, O, O]",559
4,"[turn, on, all, my, strom, wifi, switch]","[O, O, O, O, O, O, O]",408
5,[standby],[O],1281
6,"[please, start, this, show, from, the, beginning]","[O, O, O, O, O, O, O]",443
7,"[we, need, guest, wi, fi, now]","[O, O, O, O, O, O]",802
8,"[could, you, please, press, play]","[O, O, O, O, O]",814
9,"[show, me, live, tv, again]","[O, O, O, O, O]",106


In [49]:
from transformers import DistilBertTokenizerFast, AutoTokenizer, AutoModelWithLMHead
from transformers import DistilBertForTokenClassification
pretrained_path = './pretrained/distilbert-base-uncased/'
model = DistilBertForTokenClassification.from_pretrained(pretrained_path, num_labels=3)
tokenizer = DistilBertTokenizerFast.from_pretrained(pretrained_path)

In [50]:
label_all_tokens = True
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["token"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tag"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenize_and_align_labels(datasets['train'][:3])

{'input_ids': [[101, 2735, 2125, 2026, 1045, 8957, 102], [101, 3531, 5587, 2023, 3185, 2000, 2026, 5633, 102], [101, 2129, 2152, 2003, 1996, 4860, 1999, 9814, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 0, 0, 0, -100], [-100, 0, 0, 0, 0, 0, 0, 0, -100], [-100, 0, 0, 0, 0, 0, 0, 0, -100]]}

In [51]:
import numpy as np
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification

batch_size = 16
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)
label_list = datasets["train"].features[f"tag"].feature.names
data_collator = DataCollatorForTokenClassification(tokenizer)

args = TrainingArguments(
    f"exp-1",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [46]:
def predict_dataset(trainer, data):
    predictions, labels, _ = trainer.predict(data)
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

label, pred = predict_dataset(trainer, tokenized_datasets['test'])

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'sentence_id', 'tag', 'token'],
        num_rows: 1106
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'sentence_id', 'tag', 'token'],
        num_rows: 136
    })
    validtaion: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'sentence_id', 'tag', 'token'],
        num_rows: 123
    })
    train_all: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'sentence_id', 'tag', 'token'],
        num_rows: 1229
    })
})

In [ ]:
from classification_report import f1_scores, confusion_matrix
f1_scores(pred, label)

In [ ]:
confusion_matrix(pred, label, label_list)